# Set Fixed Psats

Read default yaml files, write out fixed Psats according to a prescription.

LATS:  CHLAT and SPLAT use average value.
SPLAT:  MF1_1 and MF2_1 averaged, same for MF1_2 and MF2_2

In [1]:
# See what yaml files are locally available.
!ls *.yaml

CHLAT_20210605.yaml    SAT_pole_20210605.yaml SPLAT_20210605.yaml


In [2]:
# Standard imports
import numpy as np
import yaml
import toml
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline  
plt.rcParams.update({'font.size': 18})
plt.rcParams['figure.figsize'] = [12, 20]

# bolo-calc import
from bolo import Top

In [3]:
data = {}
fixed_psats = {}
yamlfiles = {'CHLAT':'CHLAT_20210605',
             'SPLAT':'SPLAT_20210605',
             'SAT'  :'SAT_pole_20210605'}

print('Psats derived from optical power and Psat safety factor')
for tel in yamlfiles.keys(): 
    yamlfile = yamlfiles[tel]+'.yaml'
    fixed_psats[yamlfiles[tel]]={}
    dd = yaml.safe_load(open(yamlfile))
    version_date = str(dd['version']['date'])
    telescope = str(dd['version']['name'])
    data[telescope] = {}
    print('{0:8s}  Psat(pW)'.format(telescope))
    del dd['version']

    # location of atmosphere file
    dd['sim_config']['config_dir'] = '../../bolo-calc/config'

    top = Top(**dd)
    top.run()
    
    for chan in top.instrument.tables.keys():
        if chan.split('_')[0] == 'cam' :
            chname = chan.split('_')[2] + '_' + chan.split('_')[3]
            data[telescope][chname] = {}
            #
            # Grab things we're going to print and give them short names and store them in the dictionary.
            psat = top.instrument.tables[chan]['P_sat'][0]
            data[telescope][chname]['psat'] = psat
            
            print('{0:5s} : {1:8.2f}'.format(chname,psat))
    print(' ')


# how many decimals to round to.
ddd = 3

print('===========================================================')
print('LAT Psats (Average of CHLAT and SPLAT for common bands)')
chan = 'ULF_1'
fixed_psats[yamlfiles['SPLAT']][chan]= np.round(data['SPLAT'][chan]['psat'],ddd)
print('{0:5s} : {1:8.3f}'.format(chan,data['SPLAT'][chan]['psat']))
for chan in data['CHLAT'].keys():
    psat_avg = np.round((data['CHLAT'][chan]['psat'] + data['SPLAT'][chan]['psat'])/2,ddd)
    fixed_psats[yamlfiles['CHLAT']][chan]= psat_avg
    fixed_psats[yamlfiles['SPLAT']][chan]= psat_avg
    print('{0:5s} : {1:8.3f}'.format(chan,psat_avg))
    

print('======================')
print('SAT Psats (Average of _1 and _2 in MF bands)')

fixed_psats[yamlfiles['SAT']]['LF_1']= np.round(data['SAT_pole']['LF_1']['psat'],ddd)
fixed_psats[yamlfiles['SAT']]['LF_2']= np.round(data['SAT_pole']['LF_2']['psat'],ddd)

avg90 = np.round((data['SAT_pole']['MF1_1']['psat'] + data['SAT_pole']['MF2_1']['psat'])/2,ddd)
fixed_psats[yamlfiles['SAT']]['MF1_1']= avg90
fixed_psats[yamlfiles['SAT']]['MF2_1']= avg90
#
avg150 = np.round((data['SAT_pole']['MF1_2']['psat'] + data['SAT_pole']['MF2_2']['psat'])/2,ddd)
fixed_psats[yamlfiles['SAT']]['MF1_2']= avg150
fixed_psats[yamlfiles['SAT']]['MF2_2']= avg150

fixed_psats[yamlfiles['SAT']]['UHF_1']= np.round(data['SAT_pole']['UHF_1']['psat'],ddd)
fixed_psats[yamlfiles['SAT']]['UHF_2']= np.round(data['SAT_pole']['UHF_2']['psat'],ddd)

for chan in fixed_psats[yamlfiles['SAT']].keys():
    print('{0:5s} : {1:8.3f}'.format(chan,fixed_psats[yamlfiles['SAT']][chan]))     
    

Psats derived from optical power and Psat safety factor
CHLAT     Psat(pW)
LF_1  :     0.69
LF_2  :     3.63
MF_1  :     4.14
MF_2  :    12.88
UHF_1 :    37.51
UHF_2 :    55.28
 
SPLAT     Psat(pW)
ULF_1 :     0.41
LF_1  :     0.69
LF_2  :     4.30
MF_1  :     4.66
MF_2  :    11.52
UHF_1 :    29.10
UHF_2 :    39.99
 
SAT_pole  Psat(pW)
LF_1  :     1.41
LF_2  :     6.13
MF1_1 :     7.39
MF1_2 :    12.21
MF2_1 :     7.66
MF2_2 :    13.29
UHF_1 :    29.96
UHF_2 :    39.16
 
LAT Psats (Average of CHLAT and SPLAT for common bands)
ULF_1 :    0.406
LF_1  :    0.686
LF_2  :    3.965
MF_1  :    4.401
MF_2  :   12.199
UHF_1 :   33.308
UHF_2 :   47.636
SAT Psats (Average of _1 and _2 in MF bands)
LF_1  :    1.408
LF_2  :    6.133
MF1_1 :    7.520
MF2_1 :    7.520
MF1_2 :   12.750
MF2_2 :   12.750
UHF_1 :   29.959
UHF_2 :   39.164


# Save fixed psats to toml file.

In [4]:
# Write to toml file
with open('FixedPsats.toml', "w") as toml_file:
    toml.dump(fixed_psats, toml_file, encoder=toml.TomlNumpyEncoder())